In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [2]:
documents[0]

{'text': 'El Eje 4 contribuirá al repoblamiento y mejoramiento urbano del Centro Histórico de Puebla, protegiendo su patrimonio histórico cultural.',
 'section': 'Vinculación a otros instrumentos municipales',
 'question': '¿Cómo contribuirá el Eje 4 a la protección del patrimonio histórico cultural del Centro Histórico?',
 'topic': 'PMD',
 'id': '335b2f94'}

In [3]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [5]:
v = model.encode('¿Qué es el Centro Historico?')

In [6]:
len(v)

384

In [9]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "topic": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "ch-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ch-questions'})

In [8]:
from tqdm.auto import tqdm

In [10]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|████████████████████████████████████████████████| 186/186 [00:16<00:00, 11.27it/s]


In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████| 186/186 [00:04<00:00, 39.19it/s]


In [12]:
query = '¿Qué es el Centro Historico?'

In [13]:
v_q = model.encode(query)

In [39]:
def elastic_search_knn(field, vector, topic):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "topic": topic
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "topic", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [42]:
def text_vector_knn(q):
    question = q['question']
    topic = q['topic']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('text_vector', v_q, topic)

In [37]:
def question_vector_knn(q):
    question = q['question']
    course = q['topic']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [16]:
import pandas as pd

In [17]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [18]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [19]:
ground_truth[0]

{'question': '¿Qué acción ayudará al Eje 4 a proteger los elementos históricos del Centro Histórico?',
 'topic': 'PMD',
 'document': '335b2f94'}

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [44]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [45]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        v = embedding_model.encode(q['question'])
        results = search_function(q)
        # relevance = []
        # for d in results:
        #     relevance.append(d['id'] == doc_id)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [24]:
embedding_model = model

In [47]:
evaluate(ground_truth, text_vector_knn)

100%|████████████████████████████████████████████████| 370/370 [00:15<00:00, 23.55it/s]


{'hit_rate': 0.7945945945945946, 'mrr': 0.668828828828829}

In [34]:
def elastic_search_knn_combined(vector, topic):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "topic": topic
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "topic": topic
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "topic", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [48]:
def vector_combined_knn(q):
    question = q['question']
    topic = q['topic']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, topic)

In [49]:
evaluate(ground_truth, vector_combined_knn)

100%|████████████████████████████████████████████████| 370/370 [00:15<00:00, 23.70it/s]


{'hit_rate': 0.845945945945946, 'mrr': 0.7263963963963963}